In [1]:
import xarray as xr

In [2]:
import pygeode as pyg
from pygeode.tutorial import t2
import numpy as np

/home/adk33/aether/share/minconda3/envs/testpy3/lib/python3.8/site-packages/pygeode/plot/wrappers.py:560: UserWarning: Basemap functionality is unavailable.
  warnings.warn('Basemap functionality is unavailable.')


In [3]:
ds1 = xr.Dataset( 
        data_vars=dict( 
        u=(["lat", "pres"], t2.U(time=0,lon=0).squeeze()[:].transpose()), 
        olr=(["lat",], (150*pyg.cosd(t2.lat*2)+150)[:]), 
        pe=(["lat",], (-pyg.cosd(t2.lat*2))[:]),  
        v=(["pres","lat"], (-pyg.sind(t2.lat*6)*pyg.cos(t2.pres/1e3))[:].transpose()),   
        psl=(["lat",], (pyg.cosd(t2.lat*6)+1)[:]),   
        uas=(["lat",], (-pyg.cosd(2*t2.lat))[:]),    
        ), 
        coords=dict( 
        pressure=(["pres"], t2.pres[:]), 
        latitude=(["lat"], t2.lat[:]), 
        ), 
        attrs=dict(description="Weather related data."), 
) 

In [4]:
metric_dim = {'metric_dim': (['metric_dim'], np.array([0,1]))}

In [98]:
print(coords)

{'metric_dim': (['metric_dim'], array([0, 1]))}


In [9]:
data = t2.U(lon=0).squeeze() 
print(data)

<Var 'U'>:
  Shape:  (time,pres,lat)  (3650,20,31)
  Axes:
    time <ModelTime365>:  Jan 1, 2011 00:00:00 to Dec 31, 2020 00:00:00 (3650 values)
    pres <Pres>    :  1000 hPa to 50 hPa (20 values)
    lat <Lat>      :  90 S to 90 N (31 values)
  Attributes:
    {}
  Type:  SqueezedVar (dtype="float64")


In [10]:
ds2 = xr.Dataset( 
        data_vars=dict( 
        u=(["time","pres", "lat",], data[:]), 
        ), 
        coords=dict( 
        pres=(["pres"], t2.pres[:]),
        time=(["time"], t2.time[:]), 
        lat=(["lat"], t2.lat[:]), 
        ), 
) 

In [189]:
def special_mean(x):
    #print(np.shape(np.mean(x,0)))
    return (np.mean(x),np.mean)
    #return np.stack(np.mean(x), np.mean(x),axis=-1)

In [24]:
def _min_max(x):
    return np.array([np.min(x), np.max(x)])

In [189]:
>>> xr.apply_ufunc(_min_max, xds, input_core_dims=[['time', 'lat', 'lon']],
...                output_core_dims=[['min_max']])

In [179]:
def f2(x):
    print(np.shape(x))
    
    return np.mean(x)
    #return np.array([np.mean(x),np.mean(x)])

print(xr.apply_ufunc(special_mean, ds2.u, input_core_dims=[["lat","pres"]], vectorize=True))

In [25]:
metric_dim = {'metric_dim': (['metric_dim'], np.array([0,1]))}

In [44]:
ds2_mean = xr.apply_ufunc(_min_max, ds2.u, input_core_dims=[["lat","pres"]], output_core_dims=[["metrics"]], 
          dask = 'allowed',vectorize=True) 

In [45]:
ds2_mean_dataset = ds2_mean.to_dataset()

In [48]:
print(ds2_mean_dataset.assign_coords(metrics=("metrics", np.array([0,1]))))

<xarray.Dataset>
Dimensions:  (time: 3650, metrics: 2)
Coordinates:
  * time     (time) float64 0.0 1.0 2.0 3.0 ... 3.647e+03 3.648e+03 3.649e+03
  * metrics  (metrics) int64 0 1
Data variables:
    u        (time, metrics) float64 0.0 39.32 -0.4814 ... 46.14 -13.01 46.26


In [40]:
print()

<xarray.Dataset>
Dimensions:  (time: 3650, metric_dim: 2)
Coordinates:
  * time     (time) float64 0.0 1.0 2.0 3.0 ... 3.647e+03 3.648e+03 3.649e+03
Dimensions without coordinates: metric_dim
Data variables:
    u        (time, metric_dim) float64 0.0 39.32 -0.4814 ... 46.14 -13.01 46.26

In [42]:
values

<bound method Mapping.values of <xarray.Dataset>
Dimensions:   (lat: 31)
Coordinates:
    latitude  (lat) float64 -90.0 -84.0 -78.0 -72.0 ... 72.0 78.0 84.0 90.0
Dimensions without coordinates: lat
Data variables:
    u         (lat) float64 -0.9795 0.1931 3.389 8.067 ... 5.087 2.022 0.9795>